In [ ]:
# import
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import datetime
import time
import re
import duckdb
import win32com.client
import ssl
from urllib.request import urlopen
from pretty_html_table import build_table
import os

# accumulators
start_time=time.time()
df_acc=pd.DataFrame()

# links (of categories)
link='https://ushopbd.com/'
context=ssl._create_unverified_context()
html=urlopen(link, context=context)
soup_init=BeautifulSoup(html, 'lxml')
soup=soup_init.find_all("div", attrs={"class": "grid__item medium-up--one-third text-center wow fadeIn"})
urls=[link+s.find("a")['href'] for s in soup]

# scrape each category
for url in urls:
    try:
        pg = 1
        while (1):
            # soup
            link = url + '?page=' + str(pg)
            # html=requests.get(link).text
            html = urlopen(link, context=context)
            soup_init = BeautifulSoup(html, 'lxml')
            soup_cat = soup_init.find("nav", attrs={"class": "page-width breadcrumbs"})
            soup = soup_init.find("div", attrs={"class": "productList"})

            # title, brand
            sku = soup.find_all("a", attrs={"class": "grid-view-item__title"})
            title = [t.get_text() for t in sku]
            brand = [re.sub('[\W_]+', '', t.get_text().split()[0]) for t in sku]

            # weight
            quantity = []
            quantity_value = []
            quantity_unit = []

            for i in range(0, len(title)):
                bool_ind = [s[0].isdigit() and len(s) > 1 and s[len(s) - 1].isalpha() for s in title[i].split()]
                for j in range(0, len(bool_ind)):
                    if_wt_found = 0
                    if bool_ind[j] is True:
                        quantity.append(title[i].split()[j])
                        if_wt_found = 1
                        break
                if if_wt_found == 0:
                    quantity.append("")
            r = re.compile("([0-9]+)([a-zA-Z]+)")

            for w in quantity:
                if w != '':
                    quantity_value.append(float(r.match(w).group(1)))
                    quantity_unit.append(r.match(w).group(2).lower())
                else:
                    quantity_value.append(None)
                    quantity_unit.append("")

            # price
            price_bdt = soup.find_all("s", attrs={"class": "product-price__price"})
            price_bdt = [float(p.get_text()[3:].replace(',', '')) for p in price_bdt]
            price_bdt_now = soup.find_all("span", attrs={"class": "product-price__price product-price__sale"})
            price_bdt_now = [float(p.get_text()[3:].replace(',', '')) for p in price_bdt_now]

            # category
            category = soup_cat.find_all("span")
            category = category[1].get_text()

            # full data
            df = pd.DataFrame()
            df['title'] = title
            df['brand'] = brand
            df['quantity_value'] = quantity_value
            df['quantity_unit'] = quantity_unit
            df['price_bdt'] = price_bdt
            df['price_bdt_now'] = price_bdt_now
            df['category'] = category
            df_acc = df_acc.append(df)

            # next page if available
            if df.shape[0] == 0: break
            print("Data scraped from page: " + link + ", found: " + str(df.shape[0]) + " products")
            pg = pg + 1
        print()

    except:
        continue

# stats
print("Total products found: "+str(df_acc.shape[0]))
elapsed_time=time.time()-start_time
print("Elapsed time to scrape (mins): "+str(round(elapsed_time/60.00, 2)))
df_acc=df_acc.reset_index(drop=True)

# csv
df_acc.to_csv("ushopbd_all_sku_data.csv", index=False)

# analyse
qry='''
select 
  brand, 
  count(title) "SKUs (ushopbd.com)", 
  min(price_bdt_now) "min. price BDT",
  max(price_bdt_now) "max. price BDT", 
  avg((price_bdt-price_bdt_now)/price_bdt) "avg. % of discount"
from df_acc
where 1=1
  and title ilike '%shampoo%'
  and title not ilike '%glow%'
group by 1
order by 5 desc
limit 10;
'''
res_df=duckdb.query(qry).df()

# # email
# ol = win32com.client.Dispatch("outlook.application")
# olmailitem = 0x0
# newmail = ol.CreateItem(olmailitem)

# # subject, recipients
# newmail.Subject = 'Scraped & Analysed: ushopbd.com'
# newmail.To = 'shithi30@gmail.com'
# # newmail.CC='shithi30@gmail.com; maitra.shithi.aust.cse@gmail.com'
# # newmail.BCC='shithi30@gmail.com; maitra.shithi.aust.cse@gmail.com'

# # body
# newmail.HTMLbody = f'''
# Hello Bhaiya,<br><br>
# Please find below an analysis of shampoo SKUs available on ushopbd.com. Also, find all SKUs' data in the attachment.<br>
# ''' + build_table(res_df, 'blue_light') + '''
# Note that, this is an auto generated email using smtplib.<br><br>
# Thanks,<br>
# Shithi Maitra<br>
# Asst. Manager, Customer Service Excellence<br>
# Unilever BD Ltd.<br>
# '''

# # attachment(s)
# attachment = ['ushopbd_all_sku_data.csv']
# for atch in attachment:
#     newmail.Attachments.Add(os.getcwd() + '\\' + atch)

# # display.send
# # newmail.Display()
# newmail.Send()

Data scraped from page: https://ushopbd.com//collections/vaseline-pj?page=1, found: 4 products

